## Tensorflow: gradient descent y tensorboard

In [43]:
import numpy as np
import pandas as pd
import tensorflow as tf
from datetime import datetime


In [53]:
print(tf.__version__)

2.1.0


In [54]:
if tf.__version__.startswith("2."):
  import tensorflow.compat.v1 as tf
  tf.compat.v1.disable_v2_behavior()
  tf.compat.v1.disable_eager_execution()
  print("Enabled compatitility to tf1.x")

Enabled compatitility to tf1.x


#### Carga de datos

In [7]:
data = np.load('proyecto_training_data.npy')

segmentacion de datos

In [209]:
largo = len(data)
cols = ['SalePrice','OverallQual','FstFlrSF','TotRmsAbvGrd','YearBuilt','LotFrontage']

tlargo = int(largo *1)
datatrain = np.nan_to_num(data[:tlargo])

dt = pd.DataFrame(datatrain,columns=cols)
dt.describe()

,SalePrice,OverallQual,FstFlrSF,TotRmsAbvGrd,YearBuilt,LotFrontage
count,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000
mean,180921.195890,6.099315,1162.626712,6.517808,1971.267808,57.623288
std,79442.502883,1.382997,386.587738,1.625393,30.202904,34.664304
min,34900.000000,1.000000,334.000000,2.000000,1872.000000,0.000000
25%,129975.000000,5.000000,882.000000,5.000000,1954.000000,42.000000
50%,163000.000000,6.000000,1087.000000,6.000000,1973.000000,63.000000
75%,214000.000000,7.000000,1391.250000,7.000000,2000.000000,79.000000
max,755000.000000,10.000000,4692.000000,14.000000,2010.000000,313.000000


#### Regularizacion de los datos

No es esta utlizando

In [46]:
x1_reg = dt[cols[1]]/120
y_reg = dt[cols[0]] /100000

### Clase con el calculo del gradiente

In [224]:
### Objeto del modelo
class GradientDesend:
    def __init__(self,b0=0.,b1=0.):
        tf.reset_default_graph()
        self.b0 = tf.get_variable("B0",dtype=tf.float32,
                                  shape=[],initializer=tf.constant_initializer(b0))
        self.b1 = tf.get_variable("B1",dtype=tf.float32,
                                  shape=[],initializer=tf.constant_initializer(b1))
        self.error = tf.get_variable("error",dtype=tf.float32,
                                  shape=[],initializer=tf.zeros_initializer())
        
    def step(self,x,y,lr):
        # hipotesis
        yhat = tf.matmul(x,[[self.b0],[self.b1]],name="yhat") 

        error = tf.reduce_mean(tf.math.square(y - yhat) ,name="reduce_mean")/2

        grad = tf.gradients(error,[self.b0,self.b1], name = "gradients")
        
        
        local_b0 = tf.assign(self.b0, self.b0 - lr*grad[0] , name="assign_b0")
        local_b1 = tf.assign(self.b1, self.b1 - lr*grad[1] , name="assign_b1")
        local_error = tf.assign(self.error, error , name="assign_error")
        
        
        salida = tf.group(local_b0,local_b1,local_error, name="grupo")

        return salida
        
        

### Ejecucion

In [237]:

tf.reset_default_graph()
gd = GradientDesend(init_b0,init_b1)

# valores iniciales
init_b0 = 0
init_b1 = 0
lr = 0.04
iteracciones = 2000

dat_x = tf.placeholder(tf.float32,[tlargo,2],"Value_x")
dat_y = tf.placeholder(tf.float32,[tlargo],"Value_y")

step = gd.step(dat_x,dat_y,lr)

# se agrega unos a los valores de X para operarlo como matriz
nc = np.ones_like(dt[cols[1]])
x = np.hstack((np.expand_dims(dt[cols[1]],1),np.expand_dims(nc,1))) 


error_summary = tf.summary.scalar('error', gd.error)

with tf.train.MonitoredSession() as session:
    localdir = "logs/gradient_" + datetime.now().strftime("%Y%m%d-%H%M%S") + "_lr=" + str(lr) + "_b0="+ str(init_b0) + "_b1="+ str(init_b1)

    feed_dict = {dat_x:x, dat_y:dt[cols[0]]}
    writer = tf.summary.FileWriter(localdir , session.graph)
    for i in range(iteracciones):
        session.run(step,feed_dict=feed_dict)
        summary = session.run(error_summary,feed_dict=feed_dict)
        writer.add_summary(summary, i)

    
    
    b0,b1,error = session.run([gd.b0,gd.b1,gd.error],feed_dict=feed_dict)
    print("y = {}x + {} [{}]".format(b0,b1,error))


INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
y = 0.16525016725063324x + 180920.140625 [3153393664.0]


#### Modelo
<img src="GraphModel.png" >

### Escalar del error
<img src="GraphScalar.png">

## Conclusiones

Debido a que no se utilizo ningun metodo de regularizacion de los datos y la variable X seleccionada (OverallQual) su rango de valores no pasa de 10 contra los valores de venta que son cientos de miles hace dificil la obtencion de un modelo estable, sin embargo se logran observar mejoras significativas entre los diversos experimientos

El mejor modelo encontrado es con un **Learning Rate de 0.04**; arriba de eso el modelo comienza a ser divergente.  La mayoria de las veces converge en 10 iteraciones, despues los cambios son significativos pero devido a la escala de los datos son poco perceptibles

Cuando los valores iniciales de b0 y b1 se definen significativamente altos pareciera no tener una curva de cambio y se muestra mas como una linea horizontal

Aun despues de **500 iteraciones** el valor de error sigue desendiendo





##### Con los parametros 
init_b0 = 6      <-- media de los valores de x <br/>
init_b1 = 180921 <-- media de los valores de y <br/>
lr = 0.04 <br/>
iteracciones = 1000 <br/>
<br/>
Se logra el modelo siguiente
<br/>
y = 0.028576986864209175x + 180921.015625 **[Error = 3153393664.0]**




Se logra tener el mismo error con los siguintes parametros<br/>
init_b0 = 0<br/>
init_b1 = 0<br/>
lr = 0.04<br/>
iteracciones = 2000<br/><br/>
y = 0.16525016725063324x + 180920.140625 **[Error = 3153393664.0]**

<img src="GraphScalar2000.png">
